In [3]:
from keras.src.backend.jax.nn import threshold


def get_intraday_volatility(close, span=100, lookback_minutes=5):
    """
    Estimate volatility from intraday price data using exponential moving std of returns.

    Parameters:
        close (pd.Series): Intraday close prices with datetime index.
        span (int): Span for EWM standard deviation.
        lookback_minutes (int): How far back to compute returns (in minutes).

    Returns:
        pd.Series: Volatility estimates.
    """
    close = close.sort_index().drop_duplicates()

    lookback = pd.Timedelta(minutes=lookback_minutes)
    prev_index = close.index.searchsorted(close.index - lookback)

    # Remove invalid indices (e.g. index=0 means no earlier price exists)
    prev_index = prev_index[prev_index > 0]

    # Map each timestamp to its previous price `lookback_minutes` ago
    prev_dates = pd.Series(
        close.index[prev_index - 1],
        index=close.index[close.shape[0] - prev_index.shape[0]:]
    )

    # Compute returns over lookback window
    returns = close.loc[prev_dates.index] / close.loc[prev_dates.values].values - 1

    # Exponentially weighted volatility
    volatility = returns.ewm(span=span).std()

    return volatility


In [4]:
import pandas as pd
import numpy as np
from multiprocessing import cpu_count, Pool

def mp_pandas_obj(func, pdObj, numThreads=24, **kwargs):
    """
    Parallelize a function over pandas objects (e.g. index splits).

    Parameters:
    -----------
    func : function
        The function to run in parallel. It must take `molecule=` as a named argument.
    pdObj : tuple(str, pd.Index)
        Tuple with the name of the variable and the values to split into molecules.
    numThreads : int
        Number of worker processes.
    kwargs : dict
        Additional keyword arguments passed to `func`.

    Returns:
    --------
    pd.DataFrame or pd.Series
        Concatenated result of the parallel execution.
    """
    # Split the index into chunks (molecules)
    name, obj = pdObj
    parts = [part for part in np.array_split(obj, numThreads) if len(part)]

    kwargs_list = [{**kwargs, name: part} for part in parts]

    # Process in parallel
    if numThreads == 1:
        results = list(map(lambda x: func(**x), kwargs_list))
    else:
        with Pool(processes=numThreads) as pool:
            results = pool.map(lambda x: func(**x), kwargs_list)

    # Concatenate output
    if isinstance(results[0], pd.DataFrame):
        return pd.concat(results)
    elif isinstance(results[0], pd.Series):
        return pd.concat(results)
    else:
        return results


In [5]:
import pandas as pd
import numpy as np

# Simulate realistic 'close' price data
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', periods=200, freq='B')  # 200 business days
close_prices = pd.Series(np.cumprod(1 + np.random.normal(0, 0.01, size=len(dates))), index=dates)

# Simulate realistic 'events' DataFrame
event_indices = np.random.choice(dates[10:-10], size=10, replace=False)
event_indices.sort()
t1 = [dates[dates.get_loc(d) + np.random.randint(5, 15)] for d in event_indices]
trgt = np.random.uniform(0.01, 0.05, size=len(event_indices))
side = np.random.choice([1, -1], size=len(event_indices))

events = pd.DataFrame({
    't1': t1,
    'trgt': trgt,
    'side': side
}, index=event_indices)

# Define profit-taking and stop-loss thresholds
pt_sl = (1, 1)  # 1x target for both

# Molecule (subset of event indices)
molecule = events.index[:5]

# Function from above
import pandas as pd

def apply_pt_sl_on_t1(close, events, pt_sl, molecule):
    """
    Apply stop-loss (SL) and profit-taking (PT) logic on the event windows.

    Parameters:
    -----------
    close : pd.Series
        Closing price series.
    events : pd.DataFrame
        Contains 't1' (event end time), 'trgt' (target return), and 'side' (+1 for long, -1 for short).
    pt_sl : tuple(float, float)
        Tuple of profit-taking and stop-loss multipliers (e.g., (1, 1)).
    molecule : list or index
        Subset of event indices to process.

    Returns:
    --------
    pd.DataFrame
        DataFrame with the earliest PT or SL time for each event.
    """
    # Subset of events to process
    events_subset = events.loc[molecule]
    out = events_subset[['t1']].copy(deep=True)

    # Set profit-taking and stop-loss thresholds
    pt = pt_sl[0] * events_subset['trgt'] if pt_sl[0] > 0 else pd.Series(index=events.index)
    sl = -pt_sl[1] * events_subset['trgt'] if pt_sl[1] > 0 else pd.Series(index=events.index)

    # Iterate over each event
    for event_time, t1 in events_subset['t1'].fillna(close.index[-1]).items():
        price_path = close.loc[event_time:t1]                        # price path during event
        returns = (price_path / close[event_time] - 1)              # raw returns
        returns *= events_subset.at[event_time, 'side']             # apply side (long/short)

        # Find first date where SL or PT is triggered
        out.loc[event_time, 'sl'] = returns[returns < sl[event_time]].index.min()
        out.loc[event_time, 'pt'] = returns[returns > pt[event_time]].index.min()

    return out


# Apply the function
result = apply_pt_sl_on_t1(close_prices, events, pt_sl, molecule)
result # we get time that we out of the game

,t1,sl,pt
2023-02-02,2023-02-17,NaT,2023-02-13
2023-02-17,2023-03-07,NaT,2023-02-23
2023-03-13,2023-03-28,NaT,NaT
2023-03-14,2023-03-27,NaT,NaT
2023-05-16,2023-05-23,NaT,NaT


r

In [6]:
def get_events(close, t_events, pt_sl, trgt, min_ret, num_threads, t1=False, side=None, num_days=None):
    """
    Create event structure and apply the triple-barrier labeling.

    Parameters:
        close (pd.Series): Close prices.
        t_events (pd.Index): Event timestamps.
        pt_sl (tuple or float): Tuple (pt, sl) as multiples of trgt.
        trgt (pd.Series): Target returns.
        min_ret (float): Minimum return threshold.
        num_threads (int): Number of threads to use.
        t1 (pd.Series or bool): If False, vertical barrier will be computed using `num_days`.
        side (pd.Series or None): Optional side info for meta-labeling.
        num_days (int or None): Number of calendar days for vertical barrier if `t1` is False.

    Returns:
        pd.DataFrame: Events with 't1', 'trgt', and optionally 'side'.
    """
    # 1) Filter targets below threshold
    t_events = t_events.intersection(trgt.index)

    trgt = trgt.loc[t_events]
    trgt = trgt[trgt > min_ret]

    # 2) Compute vertical barrier if t1 is not provided
    if t1 is False:
        if num_days is None:
            raise ValueError("If t1 is False, num_days must be provided.")
        future_times = t_events + pd.Timedelta(days=num_days)
        t1_idx = close.index.searchsorted(future_times)
        t1_idx = t1_idx[t1_idx < close.shape[0]]
        t1 = pd.Series(close.index[t1_idx], index=t_events[:t1_idx.shape[0]])

    # 3) Define side and pt/sl levels
    if isinstance(pt_sl, (int, float)):
        pt_sl_ = (pt_sl, pt_sl)
    else:
        pt_sl_ = pt_sl[:2]  # assume it's already a tuple

    if side is None:
        side_ = pd.Series(1.0, index=trgt.index)
    else:
        side_ = side.loc[trgt.index]




    # 4) Create event DataFrame
    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1)
    events = events.dropna(subset=['trgt'])

    # 5) Apply PT/SL logic in parallel
    df0 = mp_pandas_obj(
        func=apply_pt_sl_on_t1,
        pdObj=('molecule', events.index),
        numThreads=num_threads,
        close=close,
        events=events,
        pt_sl=pt_sl_
    )

    # 6) Set final t1 as the earliest of PT/SL/vertical
    events['t1'] = df0.dropna(how='all').min(axis=1)

    # 7) Remove side if not needed
    if side is None:
        events = events.drop('side', axis=1)

    return events


In [7]:
import pandas as pd
import numpy as np

def get_bins(events, close):
    """
    Compute event returns and assign classification labels (bins).

    Parameters:
    -----------
    events : pd.DataFrame
        Must contain:
            - 't1': event end time (mandatory)
            - 'side': optional, for meta-labeling (e.g., long = 1, short = -1)
    close : pd.Series
        Series of closing prices (indexed by datetime).

    Returns:
    --------
    pd.DataFrame
        With:
            - 'ret': return from event start to t1
            - 'bin': label (+1, -1, or 0 depending on return and side)
    """
    # 1) Remove events without end time
    events = events.dropna(subset=['t1'])

    # 2) Ensure we have all required prices
    px_index = events.index.union(events['t1']).drop_duplicates()
    prices = close.reindex(px_index, method='bfill')

    # 3) Compute returns from start to end of event
    start_prices = prices.loc[events.index]
    end_prices = prices.loc[events['t1']]
    out = pd.DataFrame(index=events.index)
    out['ret'] = end_prices.values / start_prices.values - 1

    # 4) Meta-label adjustment
    if 'side' in events.columns:
        out['ret'] *= events['side']

    # 5) Assign label: sign of return
    out['bin'] = np.sign(out['ret'])

    # 6) Meta-labeling correction: only keep correct directional calls
    if 'side' in events.columns:
        out.loc[out['ret'] <= 0, 'bin'] = 0  # false positives or incorrect calls

    return out


In [8]:
def drop_labels(events, min_pct=0.05, verbose=True):
    """
    Drop labels (bins) that do not meet a minimum class proportion threshold.

    Parameters:
    -----------
    events : pd.DataFrame
        Must contain a 'bin' column with class labels.
    min_pct : float, default=0.05
        Minimum acceptable proportion of any label (e.g., 0.05 = 5%).
    verbose : bool, default=True
        Whether to print dropped label info.

    Returns:
    --------
    pd.DataFrame
        Filtered events DataFrame with rare classes removed.
    """
    while True:
        dropped = []
        while True:
            label_pct = events['bin'].value_counts(normalize=True)
            if label_pct.min() >= min_pct or label_pct.shape[0] < 2:
                break
            min_label = label_pct.idxmin()
            dropped.append((min_label, label_pct[min_label]))
            events = events[events['bin'] != min_label]
        if verbose:
            print("Dropped labels:", dropped)


    return events


In [10]:
# 1. Load data
df = pd.read_csv("Spy dataset.csv", parse_dates=['date']).set_index('date')
close = df['close'].sort_index()

# 2. Compute volatility
trgt = get_intraday_volatility(close, lookback_minutes=5).dropna()

# 3. Event timestamps
t_events = trgt.index[::5]

# 4. Get events
events = get_events(
    close=close,
    t_events=t_events,
    pt_sl=(1, 1),
    trgt=trgt,
    min_ret=0.001,
    num_threads=1,
    num_days=1
)

# 5. Label events
labeled = get_bins(events, close)

# 6. Drop rare labels
labeled = drop_labels(labeled, min_pct=0.05)

# 7. Final dataset
final = events.join(labeled, how='inner').dropna()


Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labels: []
Dropped labe

KeyboardInterrupt: 

In [11]:
final

NameError: name 'final' is not defined

In [15]:
import pandas as pd
import os

# Path to file
file_path = os.path.join(path, "SP.csv")

# Load subset of the data
df = pd.read_csv(file_path, usecols=['date', 'time', 'price', 'volume'], nrows=100_000)

# Parse datetime efficiently
df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%m/%d/%Y %H:%M:%S.%f', errors='coerce')

# Drop rows where datetime conversion failed
df.dropna(subset=['timestamp'], inplace=True)

# Drop unused columns and sort
df.drop(columns=['date', 'time'], inplace=True)
df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df = df[['price', 'volume']]

print(df.head())


                      price  volume
timestamp                          
2000-01-03 08:30:34  1496.4       0
2000-01-03 08:30:36  1496.0       0
2000-01-03 08:30:37  1495.5       0
2000-01-03 08:30:46  1495.0       0
2000-01-03 08:30:53  1495.5       0


In [20]:
daily_close = df['price'].resample('1D').last().dropna()

daily_ret = daily_close.pct_change().dropna()

threshold = daily_ret.std()

In [21]:
def apply_cumsum_filter(price_series, threshold):
    '''
    Symmetric Cumsum filter to detect large movements
    :param price_series: Series indexed by datetime
    :param threshold: Threshold for triggering events
    :return:
    list of timestemps where CUMSUM condition was triggered
    '''
    t_events = []
    s_pos, s_neg = 0.0, 0.0
    for i in range(1, len(price_series)):
        diff = price_series.iloc[i] - price_series.iloc[i-1]
        s_pos = max(0.0, s_pos + diff)
        s_neg = min(0.0, s_neg - diff)

        if s_pos > threshold:
            s_pos = 0.0
            t_events.append(price_series.index[i])
        elif s_neg < -threshold:
            s_neg = 0.0
            t_events.append(price_series.index[i])
    return pd.DatetimeIndex(t_events)


In [27]:
# Optional: Downsample to 1-minute bars
df_resampled = df['price'].resample('1min').last().dropna()
t_events = apply_cumsum_filter(df_resampled, threshold)

print("Number of event detected", len(t_events))
print(t_events[:5])


num_days = 1
t1_idx = df.index.searchsorted(t_events + pd.Timedelta(days = num_days))
t1_idx = t1_idx[t1_idx < df.shape[0]]
t1 = pd.Series(df.index[t1_idx], index = t_events[:t1_idx.shape[0]])

Number of event detected 5835
DatetimeIndex(['2000-01-03 08:31:00', '2000-01-03 08:35:00',
               '2000-01-03 08:36:00', '2000-01-03 08:44:00',
               '2000-01-03 08:45:00'],
              dtype='datetime64[ns]', freq=None)


In [28]:
def get_daily_vol(close, span=100):
    """
    Exponentially weighted daily volatility.
    """
    returns = close.pct_change()
    vol = returns.ewm(span=span).std()
    return vol

trgt = get_daily_vol(df['price'])  # using the price series


In [29]:
def get_events(close, t_events, trgt, pt_sl, t1, min_ret=0.001, side=None):
    """
    Constructs the event structure for triple-barrier method.
    """
    # Filter out too-small targets
    trgt = trgt.loc[t_events]
    trgt = trgt[trgt > min_ret]

    if side is None:
        side_ = pd.Series(1.0, index=trgt.index)
        pt_sl_ = pt_sl
    else:
        side_ = side.loc[trgt.index]
        pt_sl_ = pt_sl[:2]

    events = pd.concat({'t1': t1, 'trgt': trgt, 'side': side_}, axis=1)
    events = events.dropna(subset=['trgt'])
    return events


In [30]:
# Apply
events = get_events(
    close=df['price'],
    t_events=t_events,
    trgt=trgt,
    pt_sl=[1, 1],
    t1=t1,
    min_ret=0.001
)


KeyError: "[Timestamp('2000-01-03 08:31:00'), Timestamp('2000-01-03 08:35:00'), Timestamp('2000-01-03 08:36:00'), Timestamp('2000-01-03 08:44:00'), Timestamp('2000-01-03 08:45:00'), Timestamp('2000-01-03 08:54:00'), Timestamp('2000-01-03 08:55:00'), Timestamp('2000-01-03 08:56:00'), Timestamp('2000-01-03 08:59:00'), Timestamp('2000-01-03 09:00:00'), Timestamp('2000-01-03 09:01:00'), Timestamp('2000-01-03 09:03:00'), Timestamp('2000-01-03 09:04:00'), Timestamp('2000-01-03 09:06:00'), Timestamp('2000-01-03 09:08:00'), Timestamp('2000-01-03 09:09:00'), Timestamp('2000-01-03 09:11:00'), Timestamp('2000-01-03 09:12:00'), Timestamp('2000-01-03 09:13:00'), Timestamp('2000-01-03 09:15:00'), Timestamp('2000-01-03 09:16:00'), Timestamp('2000-01-03 09:19:00'), Timestamp('2000-01-03 09:20:00'), Timestamp('2000-01-03 09:25:00'), Timestamp('2000-01-03 09:27:00'), Timestamp('2000-01-03 09:28:00'), Timestamp('2000-01-03 09:32:00'), Timestamp('2000-01-03 09:38:00'), Timestamp('2000-01-03 09:44:00'), Ti